In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet

#set the random seeds
np.random.seed(42)
tf.random.set_seed(42)

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
#read the triplet pairs of images
train_triplets = pd.read_csv('train_triplets.txt', delimiter = ' ', names = ['A', 'B', 'C'], dtype=str) + '.jpg'
test_triplets = pd.read_csv('test_triplets.txt', delimiter = ' ', names = ['A', 'B', 'C'], dtype=str) + '.jpg'

#balance labels:
#add labels to the training dataframe
len = train_triplets.shape[0]
labels = np.ones(len)
train_triplets['labels'] = labels

#for the training data switch B and C random with 0.5 chance to balance labels
df_sample = train_triplets.sample(round(len/2))
indicies = df_sample.index
train_triplets.loc[indicies,'labels'] = 0

#dreiecks tausch:
df_B = train_triplets.loc[indicies, 'B']
train_triplets.loc[indicies,'B'] = train_triplets.loc[indicies,'C']
train_triplets.loc[indicies,'C'] = df_B

#read pre processed features
train_features = pd.read_csv('train_features_food.csv')
test_features = pd.read_csv('test_features_food.csv')

#standardization
scaler = StandardScaler()
train_features.iloc[:, 1:] = scaler.fit_transform(train_features.iloc[:, 1:])
test_features.iloc[:, 1:] = scaler.fit_transform(test_features.iloc[:, 1:])

In [3]:
def join_features(triplet,df):
    """
        for image names of dtype string this function extracts the features for the given images
        and returns the concatenated array
        
        Args:
            triplet: tupel of the image names in the triplet
            df: data frame from which we want to load the features
        returns:
            concatenated feature vector of the feature vectors of all three images
        
    """
    
    img_A = triplet[0]
    img_B = triplet[1]
    img_C = triplet[2]
    
    features_A = df[df['0'] == img_A].to_numpy()[0,1:]
    features_B = df[df['0'] == img_B].to_numpy()[0,1:]
    features_C = df[df['0'] == img_C].to_numpy()[0,1:]
    
    #concat features
    return np.concatenate([features_A,features_B,features_C])

In [4]:
num_triplets=train_triplets.shape[0]#number of triplets in the training set
num_triplets_test = test_triplets.shape[0]#number of triplets in the test set
triplets = train_triplets.to_numpy()[:,0:3]#training triplets
test_triplets = test_triplets.to_numpy()[:,0:3]#test triplets
lbs = train_triplets.to_numpy()[:,3]#labels 

In [5]:
#store the feature vector for all training triplets in a feature matrix 
train_features = np.array([join_features(triplets[i],train_features) for i in tqdm(range(num_triplets))])

100%|███████████████████████████████████| 59515/59515 [00:55<00:00, 1081.84it/s]


In [6]:
#store the feature vector for all test triplets in a feature matrix
test_features = np.array([join_features(test_triplets[i],test_features) for i in tqdm(range(num_triplets_test))])

100%|███████████████████████████████████| 59544/59544 [00:54<00:00, 1086.25it/s]


In [7]:
train_features = train_features.astype(float)
X_test_real = test_features.astype(float)

#train-test split
X_train, X_test, y_train, y_test = train_test_split(
    train_features, lbs, 
    test_size=0.1, random_state=42
)

y_train = y_train.astype(int)
X_train = X_train.astype(float)
y_test = y_test.astype(int)
X_test = X_test.astype(float)

In [27]:
#desing a model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6144, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(3072, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1536, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(768, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(384, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(192, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(96, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
#compile a model
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy')#,
        #tf.keras.metrics.Precision(name='precision'),
        #tf.keras.metrics.Recall(name='recall')
    ]
)

In [29]:
#initialize checkpoint-filepath for the best epoch 
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [30]:
#train the model
history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_test, y_test),
                    callbacks=[model_checkpoint_callback])

Epoch 1/10


2022-05-11 01:10:07.750640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


419/419 [==============================] - ETA: 0s - loss: 0.5522 - accuracy: 0.7201

2022-05-11 01:10:34.744750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


419/419 [==============================] - 28s 64ms/step - loss: 0.5522 - accuracy: 0.7201 - val_loss: 0.5434 - val_accuracy: 0.7391
Epoch 2/10
419/419 [==============================] - 25s 61ms/step - loss: 0.4411 - accuracy: 0.7950 - val_loss: 0.5134 - val_accuracy: 0.7650
Epoch 3/10
419/419 [==============================] - 26s 61ms/step - loss: 0.3597 - accuracy: 0.8430 - val_loss: 0.5246 - val_accuracy: 0.7781
Epoch 4/10
419/419 [==============================] - 26s 61ms/step - loss: 0.2876 - accuracy: 0.8797 - val_loss: 0.5311 - val_accuracy: 0.7819
Epoch 5/10
419/419 [==============================] - 26s 61ms/step - loss: 0.2218 - accuracy: 0.9091 - val_loss: 0.4970 - val_accuracy: 0.8112
Epoch 6/10
419/419 [==============================] - 26s 62ms/step - loss: 0.1756 - accuracy: 0.9312 - val_loss: 0.6255 - val_accuracy: 0.7826
Epoch 7/10
419/419 [==============================] - 26s 61ms/step - loss: 0.1409 - accuracy: 0.9449 - val_loss: 0.6594 - val_accuracy: 0.7997
Epo

In [31]:
#reload the best model
model.load_weights(checkpoint_filepath)

#print accuracy on the validation set of the best model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

  5/186 [..............................] - ETA: 3s - loss: 0.4666 - accuracy: 0.8438 

2022-05-11 01:14:45.565024: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


186/186 [==============================] - 1s 6ms/step - loss: 0.6022 - accuracy: 0.8330
Accuracy: 83.30


In [32]:
#evaluate predicitons for the test set
predictions = (model.predict(X_test_real) > 0.5).astype(int)

2022-05-11 01:14:53.889544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [33]:
#store the predictions
preds = predictions.reshape(-1).astype(int)
df_handin = pd.DataFrame({'predicitons': preds})
df_handin.to_csv('predictions_final.txt',index=False, header = False)